<a href="https://colab.research.google.com/github/YoheiFukuhara/bert_nlp/blob/main/section_2/03_simple_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルなBERTの実装
訓練済みのモデルを使用し、文章の一部の予測、及び2つの文章が連続しているかどうかの判定を行います。

## ライブラリのインストール
PyTorch-Transformers、および必要なライブラリのインストールを行います。

In [1]:
!pip install folium==0.2.1
!pip install urllib3==1.25.11
!pip install pytorch-transformers==1.2.0

     |████████████████████████████████| 69 kB 2.0 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=1a441676029a6a933d07d4c151c387e8f2195a9edd205a6e1d40ceea508d7fa0
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 127 kB 5.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
     |████████████████████████████████| 176 kB 5.6 MB/s 
     |████████████████████████████████| 132 kB 45.3 MB/s 
     |████████████████████████████████| 895 kB 24.1 MB/s 
     |████████████████████████████████| 1.2 MB 37.2 MB/s 
     |████████████████████████████████| 8.5 MB 40.2 MB/s 
    

## 文章の一部の予測
文章における一部の単語をMASKし、それをBERTのモデルを使って予測します。

In [2]:
import torch
from pytorch_transformers import BertForMaskedLM
from pytorch_transformers import BertTokenizer


text = "[CLS] I played baseball with my friends at school yesterday [SEP]"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
words = tokenizer.tokenize(text)
print(words)

100%|██████████| 231508/231508 [00:00<00:00, 3212876.89B/s]

['[CLS]', 'i', 'played', 'baseball', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


文章の一部をMASKします。

In [3]:
msk_idx = 3
words[msk_idx] = "[MASK]"  # 単語を[MASK]に置き換える
print(words)

['[CLS]', 'i', 'played', '[MASK]', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


単語を対応するインデックスに変換します。

In [5]:
word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
print(word_ids)
word_tensor = torch.tensor([word_ids])  # テンソルに変換
print(word_tensor)

[101, 1045, 2209, 103, 2007, 2026, 2814, 2012, 2082, 7483, 102]
tensor([[ 101, 1045, 2209,  103, 2007, 2026, 2814, 2012, 2082, 7483,  102]])


BERTのモデルを使って予測を行います。

In [8]:
%%time
msk_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
msk_model.cuda()  # GPU対応
msk_model.eval()  # 学習しないので評価モデルに設定

x = word_tensor.cuda()  # GPU対応
y = msk_model(x)  # 予測
print(type(y))
result = y[0] # タプル形式の0を取得
print(result.size())  # 結果の形状

_, max_ids = torch.topk(result[0][msk_idx], k=5)  # 最も大きい(確率が高い)5つの値
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())  # インデックスを単語に変換
print(result_words)

<class 'tuple'>
torch.Size([1, 11, 30522])
['basketball', 'football', 'soccer', 'baseball', 'tennis']
CPU times: user 3.82 s, sys: 229 ms, total: 4.05 s
Wall time: 4.3 s


## 文章が連続しているかどうかの判定
BERTのモデルを使って、2つの文章が連続しているかどうかの判定を行います。  
以下の関数`show_continuity`では、2つの文章の連続性を判定し、表示します。

In [9]:
from pytorch_transformers import BertForNextSentencePrediction

def show_continuity(text, seg_ids):
    words = tokenizer.tokenize(text)
    word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
    word_tensor = torch.tensor([word_ids])  # テンソルに変換

    seg_tensor = torch.tensor([seg_ids])

    nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
    nsp_model.cuda()  # GPU対応
    nsp_model.eval()

    x = word_tensor.cuda()  # GPU対応
    s = seg_tensor.cuda()  # GPU対応

    y = nsp_model(x, s)  # 予測
    result = torch.softmax(y[0], dim=1)
    print(result)  # Softmaxで確率に
    print(str(result[0][0].item()*100) + "%の確率で連続しています。")

`show_continuity`関数に、自然につながる2つの文章を与えます。

In [10]:
text = "[CLS] What is baseball ? [SEP] It is a game of hitting the ball with the bat [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

tensor([[1.0000e+00, 4.5869e-06]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
99.9995470046997%の確率で連続しています。


`show_continuity`関数に、自然につながらない2つの文章を与えます。

In [11]:
text = "[CLS] What is baseball ? [SEP] This food is made with flour and milk [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

tensor([[9.5296e-06, 9.9999e-01]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
0.000952963819145225%の確率で連続しています。
